In [6]:
predictions = []
ground_truths = []
import os

for file in os.listdir('/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/'):
    latex_file_path = os.path.join("/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/", file)
    with open(latex_file_path, 'r', encoding='utf-8') as file:
        predictions.append(file.read())
        ground_truths.append(file.read())


In [7]:
len(predictions)

4

In [9]:
!cd PDF_2_LaTeX

In [1]:
import argparse
from multiprocessing import Pool
import re
from pathlib import Path
from collections import defaultdict
from typing import List

import numpy as np

import nltk
from nltk import edit_distance
from tqdm import tqdm

import orjson

def compute_metrics(pred, gt, minlen=4):
    metrics = {}
    # if len(pred) < minlen or len(gt) < minlen:
    #     return metrics
    metrics["edit_dist"] = edit_distance(pred, gt) / max(len(pred), len(gt))
    reference = gt.split()
    hypothesis = pred.split()
    metrics["bleu"] = nltk.translate.bleu([reference], hypothesis)
    try:
        metrics["meteor"] = nltk.translate.meteor([reference], hypothesis)
    except LookupError:
        metrics["meteor"] = np.nan
    reference = set(reference)
    hypothesis = set(hypothesis)
    metrics["precision"] = nltk.scores.precision(reference, hypothesis)
    metrics["recall"] = nltk.scores.recall(reference, hypothesis)
    metrics["f_measure"] = nltk.scores.f_measure(reference, hypothesis)
    print("Inside compute_metrics")
    print("metrics", metrics)
    return metrics



In [2]:
import json
import numpy as np
from collections import defaultdict
from multiprocessing import Pool
import os


metrics = defaultdict(list)

predictions = []
ground_truths = []


# for file in os.listdir('/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/'):
#     latex_file_path = os.path.join('/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/', file)
#     with open(latex_file_path, 'r', encoding='utf-8') as latex_file:
#         latex_content = latex_file.read()
#         # ground_truth = latex_file.read()
#         predictions.append(latex_content)
#         ground_truths.append(latex_content)
        
#         # Call compute_metrics directly
#         metrics_for_file = compute_metrics(latex_content, latex_content)
        
#         for key, value in metrics_for_file.items():
#             metrics[key].append(value)
#             print(f"{key}", value)

for file in os.listdir('/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/'):
    latex_file_path = os.path.join("/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/", file)
    with open(latex_file_path, 'r', encoding='utf-8') as file:
        outputs = file.read()
        predictions.append(outputs)
        ground_truths.append(outputs)
        
        with Pool(2) as p:
            # print("inside POOL")
            _metrics = p.starmap(compute_metrics, iterable=zip(outputs, outputs))
            for m in _metrics:
                for key, value in m.items():
                    metrics[key].append(value)
            print({key: sum(values) / len(values) for key, values in metrics.items()})

scores = {}
for metric, vals in metrics.items():
    scores[f"{metric}_accuracies"] = vals
    scores[f"{metric}_accuracy"] = np.mean(vals)
try:
    print(
        f"Total number of samples: {len(vals)}, Edit Distance (ED) based accuracy score: {scores['edit_dist_accuracy']}, BLEU score: {scores['bleu_accuracy']}, METEOR score: {scores['meteor_accuracy']}"
    )
except:
    pass

    scores["predictions"] = predictions
    scores["ground_truths"] = ground_truths
    with open('/home/patidarritesh/results.json', "w") as f:
        json.dump(scores, f)

In [3]:
predictions[0][:30]

NameError: name 'predictions' is not defined

In [4]:
import json
import numpy as np
from collections import defaultdict
from multiprocessing import Pool
import os

metrics = defaultdict(list)
predictions = []
ground_truths = []

def process_file(file):
    latex_file_path = os.path.join("/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/", file)
    with open(latex_file_path, 'r', encoding='utf-8') as file:
        outputs = file.read()
        predictions.append(outputs)
        ground_truths.append(outputs)

        # Call compute_metrics directly
        metrics_for_file = compute_metrics(outputs, outputs)

        for key, value in metrics_for_file.items():
            metrics[key].append(value)

if __name__ == "__main__":
    with Pool(2) as p:
        files = os.listdir('/home/patidarritesh/PDF_2_LaTeX/pdf_2_tex/dataset/root/latex/')
        p.map(process_file, files)

    scores = {}
    for metric, vals in metrics.items():
        scores[f"{metric}_accuracies"] = vals
        scores[f"{metric}_accuracy"] = np.mean(vals)

    try:
        print(
            f"Total number of samples: {len(vals)}, Edit Distance (ED) based accuracy score: {scores['edit_dist_accuracy']}, BLEU score: {scores['bleu_accuracy']}, METEOR score: {scores['meteor_accuracy']}"
        )
    except:
        pass

    scores["predictions"] = predictions
    scores["ground_truths"] = ground_truths
    with open('/home/patidarritesh/results.json', "w") as f:
        json.dump(scores, f)


KeyboardInterrupt: 

4